In [1]:
import price
import pandas as pd
from fmp_db import fmp
pd.options.display.float_format = '{:.3f}'.format

In [2]:
db = fmp()

In [3]:
df = db.load_financials()
df['marketCap'] = df['marketCap']/100000000
df

,symbol,name,exchange,sector,industry,marketCap,numEmployees,revenue_per_share_fiveY_growth,gross_profit_margin,roe,eps_growth,div_yield,div_per_share,dps_fiveY_growth
0,A,Agilent Technologies Inc,NYSE,Healthcare,Diagnostics & Research,406.132,16400,0.619,0.534,0.167,0.306,0.006,0.748,0.669
1,AAL,American Airlines Group Inc,NASDAQ,Industrials,Airlines,155.471,113200,-0.594,-0.762,1.343,0.540,-1.000,-1.000,-1.000
2,AAP,Advance Auto Parts Inc,NYSE,Consumer Cyclical,Specialty Retail,123.820,40000,4.293,0.442,0.240,2.343,0.007,1.250,0.000
3,AAPL,Apple Inc,NASDAQ,Technology,Consumer Electronics,20794.400,100000,1.333,0.399,1.118,-0.171,0.007,0.835,0.564
4,ABBV,Abbvie Inc,NYSE,Healthcare,Drug Manufacturers—General,2025.850,48000,0.996,0.648,0.364,97.072,0.043,4.960,1.298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,Yum! Brands Inc,NYSE,Consumer Cyclical,Restaurants,357.392,38000,-1.922,0.331,-0.112,0.166,0.016,1.940,0.031
500,ZBH,Zimmer Biomet Holdings Inc,NYSE,Healthcare,Medical Devices,350.938,20000,-0.067,0.698,0.026,1437.090,0.006,0.960,0.074
501,ZBRA,Zebra Technologies Corp,NASDAQ,Technology,Communication Equipment,265.978,8800,0.522,0.461,0.310,0.142,-1.000,-1.000,0.000
502,ZION,Zions Bancorporation NA,NASDAQ,Financial Services,Banks—Regional,92.916,9682,0.599,0.255,0.077,3.708,0.024,1.360,1.997


## Minimum Fundamental Ratio Requirements

In [4]:
minMktCap = 1000
minRevGrowth = 0.3
minGPMargin = 0.2
minEPSGrowth = 0.1
minROE = 0.2
minDPSGrowth = 0.1
minDivYield = 0

In [5]:
df = df[df['marketCap'] > minMktCap]
conditions = (df['revenue_per_share_fiveY_growth'] > minRevGrowth) & (df['gross_profit_margin'] > minGPMargin) & (df['eps_growth'] > minEPSGrowth) & (df['roe'] > minROE)
df = df[conditions]
div_conds = (df['dps_fiveY_growth'] > minDPSGrowth) & (df['div_yield'] > minDivYield)
df_final = df[div_conds]

In [6]:
mom_data = {
    'symbol': [],
    'recent90D_high': [],
    'currentPrice': [],
    '90D_support': [],
    '180D_support': [],
    '360D_support': [] 
}

In [7]:
for symbol in list(df_final['symbol']):
    print(symbol)
    currPrice = db.get_current_price(symbol)
    high = price.calculate_prev_max_high(symbol, 90)
#     fifteen = high * 0.85
#     twenty = high * 0.80
#     twentyFive = high * 0.75
    
    m3 = price.calculate_prev_min_low(symbol, 90)
    m6 = price.calculate_prev_min_low(symbol, 180)
    m12 = price.calculate_prev_min_low(symbol, 360)
    
    mom_data['symbol'].append(symbol)
    mom_data['currentPrice'].append(currPrice)
    mom_data['recent90D_high'].append(high)
#     mom_data['15%_discount'].append(fifteen)
#     mom_data['20%_discount'].append(twenty)
#     mom_data['25%_discount'].append(twentyFive)
    mom_data['90D_support'].append(m3)
    mom_data['180D_support'].append(m6)
    mom_data['360D_support'].append(m12)

ABBV
AMT
AXP
CAT
DE
HD
INTC
JNJ
MRK
NKE
NVDA
ORCL
TGT
UNH


In [8]:
mom_df = pd.DataFrame(mom_data)
mom_df.set_index('symbol', inplace=True)
mom_df

,recent90D_high,currentPrice,90D_support,180D_support,360D_support
symbol,,,,,
ABBV,118.280,111.400,101.810,79.110,62.550
AMT,258.680,262.930,197.500,197.500,174.320
AXP,164.110,164.120,112.100,89.110,67.000
CAT,245.780,243.460,179.340,142.730,87.500
DE,400.340,356.710,278.950,210.180,106.140
HD,345.690,313.230,246.590,246.590,140.630
INTC,68.490,57.480,53.300,43.610,43.610
JNJ,173.650,166.200,151.470,133.650,109.160
MRK,81.430,75.870,71.720,71.720,65.250


In [9]:
buyList = []

for symbol in list(mom_df.index):
    currPrice = mom_df.loc[symbol,'currentPrice']
    if currPrice < mom_df.loc[symbol,'90D_support']:
        print(f'{symbol} \t 90D_support')
    elif currPrice < mom_df.loc[symbol, '180D_support']:
        print(f'{symbol} \t 180D_support')
    elif currPrice < mom_df.loc[symbol, '360D_support']:
        print(f'{symbol} \t 360D_support')
    else:
        print(f'{symbol} \t still expensive.. be patient!')

ABBV 	 still expensive.. be patient!
AMT 	 still expensive.. be patient!
AXP 	 still expensive.. be patient!
CAT 	 still expensive.. be patient!
DE 	 still expensive.. be patient!
HD 	 still expensive.. be patient!
INTC 	 still expensive.. be patient!
JNJ 	 still expensive.. be patient!
MRK 	 still expensive.. be patient!
NKE 	 still expensive.. be patient!
NVDA 	 still expensive.. be patient!
ORCL 	 still expensive.. be patient!
TGT 	 still expensive.. be patient!
UNH 	 still expensive.. be patient!
